## Final Project


## Task 1: Defining your Problem and Audience

✅ Deliverables

#### Problem Statement 

Christians often struggle with the Bible’s complex language, historical context, and interconnected passages. Traditional study tools lack immediacy and personalization. An LLM-powered Bible study tool addresses this by providing real-time, context-aware explanations, answering theological questions, and creating tailored resources. With interactive features like quizzes and personalized challenges, it enhances engagement and accessibility, helping believers deepen their faith effectively.

Audience:
2.6 billions of Christians in the world

## Task 2: Propose a Solution

✅ Deliverables

#### Application Framework

<img src="image/BibleStudy_Diagram.jpg" />

- LLM 
    - "gpt-4o" is used for agent reasoning
    - "gpt-4o-mini" is used in the Bible rag_chain and quiz_question_generator
- Embedding Model
    -  the embedding model fine tuned for Genesis content and based on Snowflake/snowflake-arctic-embed-l (done in Midterm)
- Orchestration
    - There are two nodes in the graph: 1 agent node, 1 action node. The agent node does some reasoning based on the user queries and determines which tool in the action node to use. The action node provides a tool set for the agent node to use which includes
        1. ai_rag_tool which answers the questions inside Bible
        2. tavily_tool which searchs the topic on Internet
        3. quiz_question_generator which generates interesting test questions based on verse range, so that this application may provide a quiz to test user's understanding on the verses 
- Vector Database
    - Qdrant as it is a popular choirc for RAG. It is open sourced, excels at performing fast, accurate, and scalable vector similarity searches.
- Monitoring
    - LangSmith trace
- User Interface
    - Chainlit as it is open sourced and ease to use


## Task 3: Dealing with the Data

✅ Deliverables
1. Describe all of your data sources and external APIs, and describe what you’ll use them for.

    - The book of Genesis is downloaded to local from https://www.vatican.va/archive/bible/genesis/documents/bible_genesis_en.html by using curl command. And then it is loaded by using langchain director loader.

2. Describe the default chunking strategy that you will use.  Why did you make this decision?

    - Instead of using RecursiveCharacterTextSplitter (which was used in Midterm),  `load_genesis_documents` function is used to load and parse HTML files containing Bible verses from the specified directory, extract verse content, chapter number and verse number from verses and create Document objects with metadata. 
    - Use same fine tuned embedding model `kcheng0816/finetuned_arctic_genesis`
    - Convert Document objects into PointStruct objects with custom IDs (based on chapter and verse numbers), verse embeddings and payloads include the verse text and metadata, and upserted into the Qdrant collection.
    - `retrieve_documents` function is used to retrieve documents from a Qdrant collection based on the input question. It first checks if the question contains a specific Bible verse reference(e.g., "Genesis 1:1-5"). If a reference is found, it retrieves the exact verses using `retrieve_verse_content` function. If no reference is found, it performs a semantic search using embeddings to find the most relevant documents.


## Task 4: Building a Quick End-to-End Prototype

✅ Deliverables

https://huggingface.co/spaces/kcheng0816/BibleStudy

### Installing Dependencies

In [5]:
!uv pip install -qU ragas==0.2.10

In [1]:
!uv pip install -qU cohere langchain_cohere langchain_huggingface

In [4]:
!uv pip install -qU sentence_transformers datasets pyarrow

In [3]:
!uv pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 beautifulsoup4 lxml

In [2]:
!uv pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### Environment Variables

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE5 - Bible Study Tool - {uuid4().hex[0:8]}"
print(os.environ["LANGCHAIN_PROJECT"])


AIE5 - Bible Study Tool - 9d8a18ba


### Data Preparation

In [ ]:
!curl https://www.vatican.va/archive/bible/genesis/documents/bible_genesis_en.html -o data/bible_genesis_en.html

In [4]:
from langchain.docstore.document import Document
import re
from bs4 import BeautifulSoup
import os

path = "data/"
book = "Genesis"
collection_name = "genesis_study"
# Parse the html file and Load Genesis documents 
def load_genesis_documents(path, book_name):
    """
    Load and parse HTML files containing Bible verses from the specified directory.
    Extracts verses from Genesis and creates Document objects with metadata.

    Args:
        path (str): The directory path containing HTML files with Bible verses.
        book_name (str): The name of the book (e.g., "Genesis").

    Returns:
        list[Document]: A list of Document objects, each containing a verse's text and metadata.
    """
    documents = []
    for file in os.listdir(path):
        if file.endswith(".html"):
            file_path = os.path.join(path, file)
            with open(file_path, "r", encoding="utf-8") as f:
                soup = BeautifulSoup(f, "html.parser")
                p_tags = soup.find_all("p", align="left")
                for p_tag in p_tags:
                    verse_texts = [content.strip() for content in p_tag.contents 
                                   if isinstance(content, str) and content.strip()]
                    for verse in verse_texts:
                        match = re.match(r"\[(\d+):(\d+)\]\s*(.*)", verse)
                        if match:
                            chapter = int(match.group(1))
                            verse_num = int(match.group(2))
                            text = match.group(3)
                            doc = Document(
                                page_content=text,
                                metadata={"book": book_name, "chapter": chapter, "verse": verse_num}
                            )
                            documents.append(doc)
    return documents

In [5]:
documents = load_genesis_documents("data/", "Genesis")
print(len(documents))
print(documents[1000:1015])


1533
[Document(metadata={'book': 'Genesis', 'chapter': 34, 'verse': 20}, page_content='So Hamor and his son Shechem came to the gate of their city and spoke to the men of their city, saying,'), Document(metadata={'book': 'Genesis', 'chapter': 34, 'verse': 21}, page_content='"These people are friendly with us; let them live in the land and trade in it, for the land is large enough for them; let us take their daughters in marriage, and let us give them our daughters.'), Document(metadata={'book': 'Genesis', 'chapter': 34, 'verse': 22}, page_content='Only on this condition will they agree to live among us, to become one people: that every male among us be circumcised as they are circumcised.'), Document(metadata={'book': 'Genesis', 'chapter': 34, 'verse': 23}, page_content='Will not their livestock, their property, and all their animals be ours? Only let us agree with them, and they will live among us."'), Document(metadata={'book': 'Genesis', 'chapter': 34, 'verse': 24}, page_content='An

### Embedding Model and Vector Store

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from qdrant_client.http.models import PointStruct
import uuid

# Initialize embeddings using a fine-tuned model from HuggingFace
# The model "kcheng0816/finetuned_arctic_genesis" is tailored for Genesis-related content, ideal for a Bible Study tool
huggingface_embeddings = HuggingFaceEmbeddings(model_name="kcheng0816/finetuned_arctic_genesis")
# This dynamically determines the vector size based on the model's output, ensuring compatibility
dimension = len(huggingface_embeddings.embed_query("test"))

# Set up an in-memory Qdrant client
# Qdrant is a vector database used here to store and retrieve embeddings efficiently in memory
client = QdrantClient(":memory:")

# Create a new collection in Qdrant to store the embeddings
# 'collection_name' (e.g., "genesis_study") names the collection; VectorParams sets the vector size and uses cosine similarity for distance measurement
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=dimension, distance=Distance.COSINE)
)

# Prepare a list of PointStruct objects for uploading to Qdrant
# Each PointStruct pairs an embedding with its document's metadata for storage and retrieval
embeddings = huggingface_embeddings.embed_documents([doc.page_content for doc in documents])
points = [
    PointStruct(
        # Generate a unique ID using UUID5 based on chapter and verse
        # This ensures consistent, reproducible IDs for each verse across different runs
        id=str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{doc.metadata['chapter']}_{doc.metadata['verse']}")),
        vector=embedding,
        payload={
            "text": doc.page_content,
            "book": doc.metadata["book"],
            "chapter": doc.metadata["chapter"],
            "verse": doc.metadata["verse"]
        }
    )
    for embedding, doc in zip(embeddings, documents)
]
client.upsert(collection_name=collection_name, points=points)

/opt/anaconda3/envs/llmops-course/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertModel were not initialized from the model checkpoint at kcheng0816/finetuned_arctic_genesis and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [7]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue, MatchAny

filter = Filter(
        must=[
            FieldCondition(key="book", match=MatchValue(value="Genesis")),
            FieldCondition(key="chapter", match=MatchValue(value=1)),
            FieldCondition(key="verse", match=MatchAny(any=[1,2,3]))
        ]
    )
search_result = client.scroll(
        collection_name="genesis_study", 
        scroll_filter=filter,
        limit=3
    )
search_result

([Record(id='25277d86-b537-5f22-9a56-3da3aee3d9b8', payload={'text': 'In the beginning when God created the heavens and the earth,', 'book': 'Genesis', 'chapter': 1, 'verse': 1}, vector=None, shard_key=None, order_value=None),
  Record(id='932ab876-7702-5073-a0a1-a3173a25231c', payload={'text': 'the earth was a formless void and darkness covered the face of the deep, while a wind from God swept over the face of the waters.', 'book': 'Genesis', 'chapter': 1, 'verse': 2}, vector=None, shard_key=None, order_value=None),
  Record(id='bae05c86-bd23-5f55-b1ae-7ce063a0f460', payload={'text': 'Then God said, "Let there be light"; and there was light.', 'book': 'Genesis', 'chapter': 1, 'verse': 3}, vector=None, shard_key=None, order_value=None)],
 None)

### Retriever

In [8]:
import re
from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter, FieldCondition, MatchValue, MatchAny
from langchain.docstore.document import Document

def parse_verse_reference(ref: str):
    """
    Parse a verse reference string into book, chapter, and a list of verse numbers.
    
    Args:
        ref (str): The verse reference, e.g., "Genesis 1:1-10".
    
    Returns:
        tuple: (book, chapter, verses) where verses is a list of integers, or None if invalid.
    """
    match = re.match(r"(\w+(?:\s\w+)?)\s(\d+):([\d,-]+)", ref)
    if not match:
        return None
    book, chapter, verse_part = match.groups()
    chapter = int(chapter)
    verses = []
    for part in verse_part.split(','):
        if '-' in part:
            start, end = map(int, part.split('-'))
            verses.extend(range(start, end + 1))
        else:
            verses.append(int(part))
    return book, chapter, verses

In [9]:
def retrieve_verse_content(verse_range: str, client: QdrantClient):
    """
    Retrieve Bible verses from Qdrant based on the specified verse range.

    Parameters:
    - verse_range (str): The verse range in the format "Book Chapter:Verses", e.g., "Genesis 1:1-5".
    - client (QdrantClient): The Qdrant client to query the database.

    Returns:
    - list[Document]: A list of Document objects containing the verse text and metadata.
    - str: An error message if the verse range is invalid or no verses are found.
    """
    # Parse the verse range into book, chapter, and verses
    parsed = parse_verse_reference(verse_range)
    if not parsed:
        return "Invalid verse range format."
    book, chapter, verses = parsed

    # Create a filter for Qdrant to match the specified book, chapter, and verses
    filter = Filter(
        must=[
            FieldCondition(key="book", match=MatchValue(value=book)),
            FieldCondition(key="chapter", match=MatchValue(value=chapter)),
            FieldCondition(key="verse", match=MatchAny(any=verses))
        ]
    )

    # Retrieve the verses from Qdrant using the filter
    search_result = client.scroll(
        collection_name=collection_name,
        scroll_filter=filter,
        limit=len(verses)
    )
    if not search_result[0]:
        return "No verses found for the specified range."
    
    # Sort the retrieved points by verse number to ensure sequential order
    sorted_points = sorted(search_result[0], key=lambda p: p.payload["verse"])

    # Create Document objects from the sorted points
    docs = [
        Document(
            page_content=p.payload["text"],
            metadata=p.payload
        )
        for p in sorted_points
    ]
    return docs

In [10]:
docs = retrieve_verse_content("Genesis 1:1-3", client)
docs
# Output: List of Document objects for Genesis 1:1, 1:2, 1:3

[Document(metadata={'text': 'In the beginning when God created the heavens and the earth,', 'book': 'Genesis', 'chapter': 1, 'verse': 1}, page_content='In the beginning when God created the heavens and the earth,'),
 Document(metadata={'text': 'the earth was a formless void and darkness covered the face of the deep, while a wind from God swept over the face of the waters.', 'book': 'Genesis', 'chapter': 1, 'verse': 2}, page_content='the earth was a formless void and darkness covered the face of the deep, while a wind from God swept over the face of the waters.'),
 Document(metadata={'text': 'Then God said, "Let there be light"; and there was light.', 'book': 'Genesis', 'chapter': 1, 'verse': 3}, page_content='Then God said, "Let there be light"; and there was light.')]

In [11]:
def retrieve_documents(question: str, client: QdrantClient):
    """
    Retrieve documents from a Qdrant collection based on the input question.

    This function first checks if the question contains a specific Bible verse reference
    (e.g., "Genesis 1:1-5"). If a reference is found, it retrieves the exact verses using
    `retrieve_verse_content`. If no reference is found, it performs a semantic search
    using embeddings to find the most relevant documents.

    Parameters:
    - question (str): The input question or query string.
    - collection_name (str): The name of the Qdrant collection to search in.
    - client (QdrantClient): The Qdrant client object used to interact with the database.

    Returns:
    - list[Document]: A list of Document objects containing the relevant verse text and metadata.
    - str: An error message if no relevant documents are found or if the verse reference is invalid.
    """
    reference_match = re.search(r"(\w+)\s?(\d+):\s?([\d,-]+)", question)
    if reference_match:
        verse_range = reference_match.group(1) + ' ' + reference_match.group(2) + ':' + reference_match.group(3)
        return retrieve_verse_content(verse_range, client)
    else:
        query_vector = huggingface_embeddings.embed_query(question)
        search_result = client.query_points(
            collection_name=collection_name,
            query=query_vector,
            limit=5,
            with_payload=True
        ).points
        if search_result:
            return [
                Document(
                    page_content=point.payload["text"],
                    metadata=point.payload
                )
                for point in search_result
            ]
        return "No relevant documents found."

In [ ]:
docs = retrieve_documents("Genesis 2:3-5", client)
docs

[Document(metadata={'text': 'So God blessed the seventh day and hallowed it, because on it God rested from all the work that he had done in creation.', 'book': 'Genesis', 'chapter': 2, 'verse': 3}, page_content='So God blessed the seventh day and hallowed it, because on it God rested from all the work that he had done in creation.'),
 Document(metadata={'text': 'These are the generations of the heavens and the earth when they were created. In the day that the LORD God made the earth and the heavens,', 'book': 'Genesis', 'chapter': 2, 'verse': 4}, page_content='These are the generations of the heavens and the earth when they were created. In the day that the LORD God made the earth and the heavens,'),
 Document(metadata={'text': 'when no plant of the field was yet in the earth and no herb of the field had yet sprung up - for the LORD God had not caused it to rain upon the earth, and there was no one to till the ground;', 'book': 'Genesis', 'chapter': 2, 'verse': 5}, page_content='when n

### Augmented

In [13]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

In [14]:
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=1,  # <-- make a request once every 1 seconds!!
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
)

chat_model = init_chat_model("gpt-4o-mini", rate_limiter=rate_limiter)

from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache
set_llm_cache(InMemoryCache())


#### RAG Chain ####

In [15]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# Factory function to create the Runnable
def create_retriever_runnable(client: QdrantClient) -> RunnableLambda:
    return RunnableLambda(lambda question: retrieve_documents(question, client))

retrieval_runnable = create_retriever_runnable(client)

def format_docs(docs):
    if isinstance(docs, str):  # Handle error message case
        return docs
    return "\n\n".join(f"Genesis {doc.metadata['chapter']}:{doc.metadata['verse']} - {doc.page_content}" for doc in docs)

rag_chain = (
    {"context": retrieval_runnable | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | RunnablePassthrough.assign(response=rag_prompt | chat_model | StrOutputParser())
)



In [16]:
import time

def test_rag_llm_cache(question):
    start_time = time.time()
    rag_chain.invoke(question)
    time_no_cache = time.time() - start_time

    start_time = time.time()
    rag_chain.invoke(question)
    time_with_cache = time.time() - start_time

    print(f"First call to LLM -- time without cache: {time_no_cache:.4f} seconds")
    print(f"Second call to LLM -- time with cache: {time_with_cache:.4f} seconds")

test_rag_llm_cache("How did GOD create the whole universe in Genesis? How many days did GOD take and what did GOD do in each day")
 # Prints the context (optional)

First call to LLM -- time without cache: 6.2720 seconds
Second call to LLM -- time with cache: 0.1089 seconds


In [17]:
response = rag_chain.invoke("Could you please explain Genesis 3:16-17?")
print(response["response"]) 

Genesis 3:16-17 describes the consequences of the actions taken by the first man and woman in the Garden of Eden, specifically their disobedience to God's command.

In Genesis 3:16, God speaks to the woman and outlines two key consequences of her actions. First, He states that He will greatly increase her pain in childbearing, indicating that childbirth will involve significant suffering. Second, He notes that while she will have a desire for her husband, he will have dominion over her, implying a change in the dynamics of their relationship where the man will hold authority.

In Genesis 3:17, God then addresses the man, telling him that because he listened to his wife and ate from the forbidden tree, the ground is cursed as a result. This curse means that he will have to work the land with great difficulty, experiencing toil and hardship in providing for himself throughout his life.

Together, these verses illustrate the repercussions of human disobedience, affecting both men and wome

#### ai_rag_tool for answering bible questions ####

In [19]:
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

def format_contexts(docs):
    return "\n\n".join(docs) if isinstance(docs, list) else docs

@tool
def ai_rag_tool(question: str):
    """Useful for when you need to answer questions about Bible """
    response = rag_chain.invoke(question)
    return {
        "message": [HumanMessage(content=response["response"])],
        "context": format_contexts(response["context"])
    }

In [ ]:
result = ai_rag_tool("How did GOD create the whole universe in Genesis? How many days did GOD take and what did GOD do in each day?")
print(result)

#### tavily_tool for searching more information on Internet ####

In [20]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)   

#### Bible Quiz Question Generator ####

In [ ]:
def _generate_quiz_question(verse_range: str, client: QdrantClient):
    docs = retrieve_verse_content(verse_range, client)
    if isinstance(docs, str):
        return {"error": docs}
    
    # Randomly select a subset of verses if the range has more than 3 verses
    num_verses = len(docs)
    if num_verses > 3:
        subset_size = random.randint(1, 3)
        start_idx = random.randint(0, num_verses - subset_size)
        selected_docs = docs[start_idx : start_idx + subset_size]
    else:
        selected_docs = docs
    
    verse_content = "\n".join(
        f"{doc.metadata['book']} {doc.metadata['chapter']}:{doc.metadata['verse']} - {doc.page_content}"
        for doc in selected_docs
    )
    
    quiz_prompt = ChatPromptTemplate.from_template(
        "Based on the following Bible verse(s), generate a multiple-choice quiz question with 4 options (A, B, C, D) "
        "and indicate the correct answer:\n\n"
        "{verse_content}\n\n"
        "Format your response as follows:\n"
        "Question: [Your question here]\n"
        "A: [Option A]\n"
        "B: [Option B]\n"
        "C: [Option C]\n"
        "D: [Option D]\n"
        "Correct Answer: [Letter of correct answer]\n"
        "Explanation: [Brief explanation of why the answer is correct]\n"
    )
    
    # Use a higher temperature for more diverse question generation
    chat_model_with_temp = chat_model.bind(temperature=0.9)
    response = (quiz_prompt | chat_model_with_temp).invoke({"verse_content": verse_content})
    
    response_text = response.content.strip()
    lines = response_text.split("\n")
    question = ""
    options = {}
    correct_answer = ""
    explanation = ""
    for line in lines:
        line = line.strip()
        if line.startswith("Question:"):
            question = line[len("Question:"):].strip()
        elif line.startswith(("A:", "B:", "C:", "D:")):
            key, value = line.split(":", 1)
            options[key.strip()] = value.strip()
        elif line.startswith("Correct Answer:"):
            correct_answer = line[len("Correct Answer:"):].strip()
        elif line.startswith("Explanation:"):
            explanation = line[len("Explanation:"):].strip()
    
    return {
        "quiz_question": question,
        "options": options,
        "correct_answer": correct_answer,
        "explanation": explanation,
        "verse_range": verse_range,
        "verse_content": verse_content
    }

In [ ]:
question = _generate_quiz_question("Genesis 1:1-5", client)
question

In [22]:
import json
from functools import partial

generate_quiz_question_tool = partial(_generate_quiz_question, client=client)

@tool
def quiz_question_generator(verse_range: str):
    """Generate a quiz question based on the content of the specified verse range."""
    quiz_data = generate_quiz_question_tool(verse_range)
    return json.dumps(quiz_data)

### Tool Belt

In [23]:
tool_belt = [
    ai_rag_tool,
    tavily_tool,
    quiz_question_generator
]

### LangGraph Agent

### Binding the tools to the LLM

In [24]:
llm = init_chat_model("gpt-4o", temperature=0, rate_limiter=rate_limiter)
llm_with_tools = llm.bind_tools(tool_belt)

from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache
set_llm_cache(InMemoryCache())

### Define the State

In [25]:
from typing import Optional, List, Annotated, TypedDict
from langchain_core.messages import AnyMessage
from langchain_core.documents import Document
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    in_quiz: bool
    quiz_question: Optional[dict]
    verse_range: Optional[str]
    quiz_score: int
    quiz_total: int
    waiting_for_answer: bool

In [26]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage

response = llm_with_tools.invoke([AIMessage(content=f"Generate quiz question for Genesis 1:1-10")])
response


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_X4ff3GAzeCOVAVdW2SvmEQLy', 'function': {'arguments': '{"verse_range":"Genesis 1:1-10"}', 'name': 'generate_quiz_question'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 148, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-99c9cee6-2612-45f5-af7a-03db477dc2a9-0', tool_calls=[{'name': 'generate_quiz_question', 'args': {'verse_range': 'Genesis 1:1-10'}, 'id': 'call_X4ff3GAzeCOVAVdW2SvmEQLy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 148, 'output_tokens': 24, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 

In [28]:
import time

def test_llm_cache(question):
    start_time = time.time()
    llm_with_tools.invoke([AIMessage(content=question)])
    time_no_cache = time.time() - start_time

    start_time = time.time()
    llm_with_tools.invoke([AIMessage(content=question)])
    time_with_cache = time.time() - start_time

    print(f"First call to LLM -- time without cache: {time_no_cache:.4f} seconds")
    print(f"Second call to LLM -- time with cache: {time_with_cache:.4f} seconds")

test_llm_cache("Could you please explain Genesis 3:15-16?")
 # Prints the context (optional)

First call to LLM -- time without cache: 0.8739 seconds
Second call to LLM -- time with cache: 0.0010 seconds


In [29]:
# System message
system_message = SystemMessage(content="""You are a Bible study assistant. You can answer questions about the Bible, search the internet for related information, or generate quiz questions based on specific verse ranges.

- Use the 'ai_rag_tool' to answer questions about the Bible.
- Use the 'tavily_tool' to search the internet for additional information.
- Use the 'quiz_question_generator' tool when the user requests to start a quiz on a specific verse range, such as 'start quiz on Genesis 1:1-10'.

When the user requests a quiz, extract the verse range from their message and pass it to the 'quiz_question_generator' tool.""")

#### Agent Function ####

In [30]:
#Agent function
def call_mode(state):
    """
    Manage the conversation flow of the Bible Study Tool, focusing on quiz mode and regular interactions.

    This function determines the next action in the conversation based on the user's input and the current state.
    It handles quiz mode (processing answers, continuing or ending the quiz) and transitions to or from regular
    question-answering mode. It also processes tool calls, such as starting a quiz, and delegates non-quiz queries
    to a language model.

    Parameters:
    - state (dict): The current state of the conversation, containing messages, quiz status, and other data.

    Returns:
    - dict: An updated state dictionary with new messages and modified quiz-related fields as needed.
    """
    last_message = state["messages"][-1]
    
    if state.get("in_quiz", False):
        if state.get("waiting_for_answer", False):
            # Process the user's answer
            quiz_data = state["quiz_question"]
            user_answer = last_message.content.strip().upper()
            correct_answer = quiz_data["correct_answer"]
            new_quiz_total = state["quiz_total"] + 1
            if user_answer == correct_answer:
                new_quiz_score = state["quiz_score"] + 1
                feedback = f"Correct! {quiz_data['explanation']}"
            else:
                new_quiz_score = state["quiz_score"]
                feedback = f"Incorrect. The correct answer is {correct_answer}. {quiz_data['explanation']}"
            return {
                "messages": [
                    AIMessage(content=feedback),
                    AIMessage(content="Would you like another question? Type 'Yes' to continue or 'No' to end the quiz.")
                ],
                "quiz_total": new_quiz_total,
                "quiz_score": new_quiz_score,
                "waiting_for_answer": False,
                "quiz_question": state["quiz_question"],
                "in_quiz": True,
                "verse_range": state["verse_range"]
            }
        else:
            # Handle the user's decision to continue or stop the quiz
            user_input = last_message.content.strip().lower()
            if user_input == "yes":
                # Generate a new quiz question
                verse_range = state["verse_range"]
                quiz_data_str = quiz_question_generator(verse_range)
                quiz_data = json.loads(quiz_data_str)
                question = quiz_data["quiz_question"]
                options = "\n".join([f"{k}: {v}" for k, v in quiz_data["options"].items()])
                verse_content = quiz_data["verse_content"]
                message_to_user = (
                    f"Based on the following verse(s):\n\n{verse_content}\n\n"
                    f"Here's your quiz question:\n\n{question}\n\n{options}\n\n"
                    "Please select your answer (A, B, C, or D)."
                )
                return {
                    "messages": [AIMessage(content=message_to_user)],
                    "quiz_question": quiz_data,
                    "waiting_for_answer": True,
                    "quiz_total": state["quiz_total"],
                    "quiz_score": state["quiz_score"],
                    "in_quiz": True,
                    "verse_range": state["verse_range"]
                }
            elif user_input == "no":
                # End the quiz and provide a summary
                score = state["quiz_score"]
                total = state["quiz_total"]
                continue_message = "Ask me anything about Genesis or type 'start quiz on <verse range>' (e.g., 'start quiz on Genesis 1:1-5') for a trivia challenge."
                if total > 0:
                    percentage = (score / total) * 100
                    if percentage == 100:
                        feedback = "Excellent! You got all questions correct. Please continue your Bible study!"
                    elif percentage >= 80:
                        feedback = "Great job! You have a strong understanding. Please continue your Bible study!"
                    elif percentage >= 50:
                        feedback = "Good effort! Keep practicing to improve. Please continue your Bible study!"
                    else:
                        feedback = "Don’t worry, keep your Bible studying and you’ll get better!"
                    summary = f"You got {score} out of {total} questions correct. {feedback} \n\n {continue_message}"
                else:
                    summary = "No questions were attempted."
                return {
                    "messages": [AIMessage(content=summary)],
                    "in_quiz": False,
                    "quiz_question": None,
                    "verse_range": None,
                    "quiz_score": 0,
                    "quiz_total": 0,
                    "waiting_for_answer": False
                }
            else:
                # Handle invalid input
                return {
                    "messages": [AIMessage(content="Please type 'Yes' to continue or 'No' to end the quiz.")],
                    "quiz_total": state["quiz_total"],
                    "quiz_score": state["quiz_score"],
                    "waiting_for_answer": False,
                    "quiz_question": state["quiz_question"],
                    "in_quiz": True,
                    "verse_range": state["verse_range"]
                }
    
    # Handle starting the quiz or other tool calls
    if len(state["messages"]) >= 2 and isinstance(last_message, ToolMessage):
        prev_message = state["messages"][-2]
        if isinstance(prev_message, AIMessage) and prev_message.tool_calls:
            tool_call = prev_message.tool_calls[0]
            if tool_call["name"] == "quiz_question_generator":
                # Start the quiz
                quiz_data_str = last_message.content
                quiz_data = json.loads(quiz_data_str)
                verse_range = quiz_data["verse_range"]
                question = quiz_data["quiz_question"]
                options = "\n".join([f"{k}: {v}" for k, v in quiz_data["options"].items()])
                verse_content = quiz_data["verse_content"]
                message_to_user = (
                    f"Based on the following verse(s):\n\n{verse_content}\n\n"
                    f"Here's your quiz question:\n\n{question}\n\n{options}\n\n"
                    "Please select your answer (A, B, C, or D)."
                )
                return {
                    "messages": [AIMessage(content=message_to_user)],
                    "in_quiz": True,
                    "verse_range": verse_range,
                    "quiz_score": 0,
                    "quiz_total": 0,
                    "quiz_question": quiz_data,
                    "waiting_for_answer": True
                }
    
    # Process regular questions or commands
    messages = [system_message] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}


In [31]:
from langgraph.prebuilt import ToolNode
tool_node = ToolNode(tool_belt)

In [32]:
from langgraph.graph import END
#edge function
def should_continue(state):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "action"
    return END

### Building the graph

In [33]:
from langgraph.graph import StateGraph, END
from IPython.display import Image, display

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_mode)
uncompiled_graph.add_node("action", tool_node)

uncompiled_graph.set_entry_point("agent")

uncompiled_graph.add_conditional_edges("agent", should_continue)
uncompiled_graph.add_edge("action", "agent")

compiled_graph = uncompiled_graph.compile()


In [34]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="How many days did GOD take on creation the whole universe?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
            print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sGb7YDbb41PSJdJaZVzt5ojz', 'function': {'arguments': '{"question":"How many days did God take to create the universe according to the Bible?"}', 'name': 'ai_rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 279, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-df36c845-5b97-4d7a-853c-4e5a6022d318-0', tool_calls=[{'name': 'ai_rag_tool', 'args': {'question': 'How many days did God take to create the universe according to the Bible?'}, 'id': 'call_sGb7YDbb41PSJdJaZVzt5ojz', 'type': 'tool_call'}], usage_

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Could you search a map of Abraham's journey on Internet?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
            print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])
        print("\n\n")

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="start quiz on Genesis 1:1-10?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
            print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])
        print("\n\n")

In [ ]:
display(Image(compiled_graph.get_graph(xray=True).draw_mermaid_png()))